In [6]:
import pandas as pd
from datetime import datetime, timedelta
from iexfinance.stocks import get_historical_data
import iexfinance as iex
from iexfinance.refdata import get_symbols

In [7]:
# Use the `get_symbols` function from the iexfinance SDK to fetch available stock tickers
tickers=pd.DataFrame(get_symbols())
tickers.head()

IEXAuthenticationError: The IEX Cloud API key must be provided either through the token variable or through the environmental variable IEX_TOKEN.

8ce46d56429c451c9d7f30335308ccee


In [2]:
list_of_tickers = ["ACB.TO", "GWPH", "CRON", "CGC", "TLRY", "APHA"]
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for "SPY" and "AGG"
df = get_historical_data(list_of_tickers, start_date, end_date,  output_format='pandas')
df.head()

IEXAuthenticationError: The IEX Cloud API key must be provided either through the token variable or through the environmental variable IEX_TOKEN.

In [ ]:
## Cleaning the Data
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head()

In [8]:
import os
os.getenv('IEX_TOKEN')

'pk_1e365e4ef8d942a5be9c5687d209a083'